In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [6]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['g','recall','precision','pf']
    for goal in goals:
        count = 0
        count_not = 0
        count_yes = 0
        precentage = 0
        score = []
        score_med = []
        cluster_info = {}
        for cluster in clusters:
            df = pd.read_csv(cluster + '/1385_LR_bellwether_' + goal + '.csv')
            counts = []
            med_count = []
            for row in range(df.shape[0]):
                j = df.iloc[row].values[1:]
                j_med = np.median(j)
                if goal != 'pf':
                    value = sum(i >= 0.66 for i in j)
                else:
                    value = sum(i <= 0.33 for i in j)
                med_count.append(j_med)
                counts.append(value)
            df['>=0.6'] = counts
            precentage = max(counts)/df.shape[0]
            x = int(cluster.rsplit('/',1)[1])
            #score.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),max(counts),round(np.mean(counts),2),df.iloc[counts.index(max(counts)),0]])
            if goal == 'pf':
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.62),counts[med_count.index(min(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(min(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])     
            elif goal == 'g':
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.16),counts[med_count.index(max(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(max(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])
            else:
                score_med.append([int(cluster.rsplit('/',1)[1]),round(df.shape[0]*0.33),counts[med_count.index(max(med_count))],round(np.mean(counts),2),max(med_count),df.iloc[med_count.index(max(med_count)),0],size[int(cluster.rsplit('/',1)[1])]])
            count += max(counts)
        score_df = pd.DataFrame(score_med, columns = ['id','expected','max_bell','avge','median','bellwether','size'])
        score_df = score_df.sort_values('id')
        score_df.to_csv(data_source1 + '/bellwether_' + goal + '_2.csv')
        level_1_bellwethers = {}
        for cluster in cluster_structure[2].keys():
            if cluster_structure[2][cluster] not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster_structure[2][cluster]] = []
            level_1_bellwethers[cluster_structure[2][cluster]].append(score_df[score_df['id'] == cluster].bellwether.values[0])
        score_med = []
        for key in  level_1_bellwethers.keys():
            sub_cluster_bellwethers = level_1_bellwethers[key]
            bell = birch_bellwether.bellwether(path,df_train)
            final_score = bell.bellwether(sub_cluster_bellwethers,sub_cluster_bellwethers)
            _results = {}
            for s_project in final_score.keys():
                if s_project not in _results.keys():
                    _results[s_project] = {}
                for d_projects in final_score[s_project].keys():
                    if goal == 'g':
                        _goal = 'g-score'
                    else:
                        _goal = goal
                    _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
            _df_results = pd.DataFrame.from_dict(_results, orient='index')
            _df_results = _df_results.to_csv('temp'  + str(fold) + '.csv')
            _df_results = pd.read_csv('temp'  + str(fold) + '.csv')
            med_count = []
            counts = []
            for row in range(_df_results.shape[0]):
                j = _df_results.iloc[row].values[1:]
                j_med = np.median(j)
                if goal != 'pf':
                    value = sum(i >= 0.66 for i in j)
                else:
                    value = sum(i <= 0.33 for i in j)
                med_count.append(j_med)
                counts.append(value)
            print(_df_results)
            if goal == 'pf':
                score_med.append([key,counts[med_count.index(min(med_count))],round(np.mean(counts),2),
                                  max(med_count),_df_results.iloc[med_count.index(min(med_count)),0]])     
            elif goal == 'g':
                score_med.append([key,counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                  max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
            else:
                score_med.append([key,counts[med_count.index(max(med_count))],round(np.mean(counts),2),
                                  max(med_count),_df_results.iloc[med_count.index(max(med_count)),0]])
        score_df = pd.DataFrame(score_med, columns = ['id','max_bell','avge','median','bellwether'])
        score_df = score_df.sort_values('id')
        score_df.to_csv(data_source1 + '/bellwether_' + goal + '_1.csv')
    return cluster_structure,df_train,cluster_tree
    

In [7]:
for i in range(0,2):
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    cluster_structure,df_train,cluster_tree = find_bellwether(data_source1,clusters,path,i)

qmmp.csv
zscreen.csv
arcallians.csv
pcsx2.csv
ng4j.csv
       Unnamed: 0  qmmp.csv  zscreen.csv  arcallians.csv  pcsx2.csv  ng4j.csv
0  arcallians.csv      0.48         0.59            0.74       0.61      0.73
1        ng4j.csv      0.31         0.62            0.48       0.51      0.80
2       pcsx2.csv      0.52         0.68            0.06       0.71      0.70
3        qmmp.csv      0.67         0.62            0.38       0.65      0.66
4     zscreen.csv      0.49         0.68            0.31       0.68      0.67
pfaedit.csv
dvd-create.csv
dspace-gsoc.csv
openmalaria.csv
google-caja.csv
maya-work-in-progress.csv
clamav.csv
libarchive.csv
ftm.csv
nativeclient.csv
                  Unnamed: 0  pfaedit.csv  dvd-create.csv  dspace-gsoc.csv  \
0                 clamav.csv         0.61            0.67             0.42   
1            dspace-gsoc.csv         0.45            0.35             0.68   
2             dvd-create.csv         0.65            0.80             0.43   
3            

ngl.csv
simplewebservices.csv
gebr.csv
biopax.csv
wikipediardware.csv
              Unnamed: 0  ngl.csv  simplewebservices.csv  gebr.csv  \
0             biopax.csv     0.88                   1.00      0.84   
1               gebr.csv     0.96                   0.90      0.65   
2                ngl.csv     0.65                   0.69      0.03   
3  simplewebservices.csv     0.99                   0.67      1.00   
4    wikipediardware.csv     0.83                   0.99      0.68   

   biopax.csv  wikipediardware.csv  
0        0.75                 0.84  
1        1.00                 0.82  
2        0.49                 0.03  
3        0.32                 1.00  
4        0.45                 0.79  
pfaedit.csv
planeshift.csv
enlightenment.csv
psycle.csv
itext.csv
hydrogen.csv
frogatto.csv
st-filteredstatistics.csv
sylpheed-claws.csv
bibletime.csv
                  Unnamed: 0  pfaedit.csv  planeshift.csv  enlightenment.csv  \
0              bibletime.csv         0.91            0.9

jajuk.csv
mplayer-ce.csv
jfreereport.csv
metacosm.csv
turbotrader-bos.csv
            Unnamed: 0  jajuk.csv  mplayer-ce.csv  jfreereport.csv  \
0            jajuk.csv       0.81            0.48             0.14   
1      jfreereport.csv       0.71            0.68             0.25   
2         metacosm.csv       0.70            0.45             0.17   
3       mplayer-ce.csv       0.48            0.58             0.14   
4  turbotrader-bos.csv       0.71            0.46             0.16   

   metacosm.csv  turbotrader-bos.csv  
0          0.47                 0.35  
1          0.70                 0.53  
2          0.77                 0.37  
3          0.50                 0.38  
4          0.86                 0.60  
pfaedit.csv
dvt.csv
theresa.csv
modellus.csv
google-caja.csv
autowikibrowser.csv
riff-evolve.csv
galaxium.csv
xamj.csv
xmoto.csv
            Unnamed: 0  pfaedit.csv  dvt.csv  theresa.csv  modellus.csv  \
0  autowikibrowser.csv         0.69     0.35         0.13          

autofac.csv
exordium.csv
jcae.csv
mycila.csv
phonon-vlc-mplayer.csv
taokgame.csv
atomsite.csv
taverna.csv
xbg.csv
apolloplayer.csv
bibedt.csv
biodwh.csv
geogebra.csv
jstock.csv
e-bio-flow.csv
                Unnamed: 0  sabbu.csv  psrchive.csv  findbugs.csv  nassp.csv  \
0         apolloplayer.csv       0.61          0.17          0.40       0.23   
1             atomsite.csv       0.71          0.19          0.30       0.23   
2              autofac.csv       0.76          0.28          0.44       0.25   
3               bibedt.csv       0.52          0.17          0.38       0.25   
4               biodwh.csv       0.77          0.26          0.38       0.18   
5            chrysalis.csv       0.69          0.36          0.55       0.34   
6           e-bio-flow.csv       0.85          0.17          0.53       0.00   
7             exordium.csv       0.74          0.25          0.46       0.28   
8             findbugs.csv       0.74          0.25          0.59       0.27   
9       

log4net.csv
jpaul.csv
sblim.csv
jchassis.csv
retromenu.csv
mecat.csv
miranda-icq.csv
bionote.csv
quark.csv
customsagetv.csv
mediaportal.csv
gisgraphy.csv
piccolo2d.csv
haggle.csv
mevenide.csv
           Unnamed: 0  runuomondains.csv  log4net.csv  jpaul.csv  sblim.csv  \
0         bionote.csv               0.39         0.29       0.17       0.73   
1    customsagetv.csv               0.12         0.16       0.02       0.33   
2       gisgraphy.csv               0.51         0.19       0.20       0.24   
3          haggle.csv               0.63         0.41       0.24       0.36   
4        jchassis.csv               0.00         0.00       0.00       0.00   
5           jpaul.csv               0.31         0.24       0.34       0.05   
6         log4net.csv               0.68         0.12       0.27       0.08   
7           mecat.csv               0.02         0.07       0.02       0.05   
8     mediaportal.csv               0.41         0.12       0.05       0.18   
9        mevenide.

...

columba.csv
hibernate.csv
lphant.csv
      Unnamed: 0  raygina.csv  columba.csv  hibernate.csv  lphant.csv
0    columba.csv         0.66         0.65           0.62        0.58
1  hibernate.csv         0.40         0.38           0.59        0.70
2     lphant.csv         0.59         0.68           0.57        0.69
3    raygina.csv         0.65         0.61           0.55        0.53
triplea.csv
gnome-jabber.csv
pamguard.csv
web-cat.csv
springside.csv
open-media-library.csv
glas.csv
galaxium.csv
vienna-add-in.csv
owlib.csv
botsnscouts.csv
autofac.csv
                Unnamed: 0  triplea.csv  gnome-jabber.csv  pamguard.csv  \
0              autofac.csv         0.52              0.22          0.26   
1          botsnscouts.csv         0.55              0.40          0.60   
2             galaxium.csv         0.72              0.39          0.56   
3                 glas.csv         0.36              0.43          0.10   
4         gnome-jabber.csv         0.71              0.57          0

m-m-m.csv
coot.csv
magicwars.csv
unicase.csv
power-architect.csv
simplewebservices.csv
              Unnamed: 0  calcubetimer.csv  m-m-m.csv  coot.csv  \
0       calcubetimer.csv              0.84       0.35      0.98   
1               coot.csv              0.50       0.82      0.75   
2              m-m-m.csv              0.95       0.78      0.30   
3          magicwars.csv              0.98       0.84      1.00   
4    power-architect.csv              0.77       0.67      1.00   
5  simplewebservices.csv              0.91       0.88      0.90   
6            unicase.csv              0.46       0.80      0.95   

   magicwars.csv  unicase.csv  power-architect.csv  simplewebservices.csv  
0           0.60         0.67                 0.72                   0.09  
1           0.59         0.71                 0.43                   0.46  
2           0.78         0.72                 0.54                   0.83  
3           0.66         0.92                 0.74                   0.8

bochs.csv
fontforge.csv
modellus.csv
phonon-vlc-mplayer.csv
openmeetings.csv
bdbbasic.csv
wow-qrsk.csv
konversation.csv
sigil.csv
open-ms.csv
migen.csv
bibedt.csv
qcad2.csv
turbotrader-bos.csv
jcae.csv
genoviz.csv
emulemorph.csv
dataobjectsdotnet.csv
skia.csv
omseek.csv
                Unnamed: 0  bochs.csv  fontforge.csv  modellus.csv  \
0             bdbbasic.csv       1.00           0.45          0.38   
1               bibedt.csv       0.59           0.70          0.81   
2                bochs.csv       0.59           0.48          0.55   
3    dataobjectsdotnet.csv       0.49           0.50          0.42   
4           emulemorph.csv       0.61           0.40          0.62   
5            fontforge.csv       0.42           0.72          0.60   
6              genoviz.csv       0.92           0.50          0.90   
7                 jcae.csv       0.75           0.67          0.58   
8         konversation.csv       0.40           0.47          0.61   
9                migen.csv   

k3d.csv
gtad.csv
yaprm.csv
       Unnamed: 0  bloodycore.csv  k3d.csv  gtad.csv  yaprm.csv
0  bloodycore.csv            0.27     0.29      0.00       0.56
1        gtad.csv            0.29     0.28      0.14       0.52
2         k3d.csv            0.20     0.48      0.05       0.49
3       yaprm.csv            0.22     0.08      0.03       0.67
apertium.csv
mvdsv.csv
chaosrts.csv
web-cat.csv
gogoego.csv
forester-atv.csv
customsagetv.csv
lite.csv
nova.csv
tokratan.csv
mediaportal.csv
snakeyaml.csv
          Unnamed: 0  apertium.csv  mvdsv.csv  chaosrts.csv  web-cat.csv  \
0       apertium.csv          0.57       0.14          0.84         0.45   
1       chaosrts.csv          0.32       0.09          0.97         0.43   
2   customsagetv.csv          0.27       0.40          0.88         0.57   
3   forester-atv.csv          1.00       0.00          0.84         0.00   
4        gogoego.csv          0.37       0.13          0.89         0.41   
5           lite.csv          0.38       0

jsidplay2.csv
jchassis.csv
servicestack.csv
mecat.csv
log4net.csv
avisynth2.csv
         Unnamed: 0  freedom-erp.csv  jsidplay2.csv  jchassis.csv  \
0     avisynth2.csv             0.01           0.00          0.02   
1   freedom-erp.csv             0.33           0.10          0.02   
2      jchassis.csv             0.00           0.05          0.06   
3     jsidplay2.csv             0.94           0.20          0.23   
4       log4net.csv             0.27           0.05          0.00   
5         mecat.csv             0.03           0.00          0.33   
6  servicestack.csv             0.03           0.00          0.00   

   servicestack.csv  mecat.csv  log4net.csv  avisynth2.csv  
0              0.15       0.07         0.02           0.03  
1              0.52       0.05         0.20           0.03  
2              0.00       0.00         0.00           0.00  
3              0.46       1.00         0.31           0.08  
4              0.22       0.07         0.11           0.18  
5